In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import threading
import time

import tensorflow as tf
import numpy as np

from embedding import gen_word2vec as word2vec

In [2]:
sess = tf.InteractiveSession()

In [3]:
# practice tf.reduce_sum, tf.matmul

tmp = [[1,2,1],[2,3,1]]

print tmp
print tf.reduce_sum(tmp).eval()
print tf.reduce_sum(tmp, 0).eval()
print tf.reduce_sum(tmp, 1).eval()
print "="*30

a = np.ones([2,4])
b = np.ones([2,4])
print tf.matmul(a, b, transpose_b=True).eval()

SyntaxError: invalid syntax (<ipython-input-3-7041ef759ebc>, line 5)

In [4]:
embed_dim = 200
num_neg_samples = 100

epochs_to_train = 5
learning_rate = 0.025
batch_size = 16
window_size = 5
min_count = 5
subsample = 1e-3

# words : list of words including 'UNK'
# counts : list of counts of words
# words_per_epoch
# current_epoch : start from 0
(words, counts, words_per_epoch, current_epoch, total_words_processed, x, y) = \
    word2vec.skipgram(
        filename = 'text8',
        batch_size = batch_size,
        window_size = window_size,
        min_count = min_count,
        subsample = subsample
    )

(vocab_words, vocab_counts, words_per_epoch) = sess.run([words, counts, words_per_epoch])

In [5]:
vocab_size = len(vocab_words)

id2word = vocab_words
word2id = {}

for i, w in enumerate(id2word):
    word2id[w] = i

print("Vocab size: ", vocab_size - 1, " + UNK")
print("Words per epoch: ", words_per_epoch)

global_step = tf.Variable(0, name="global_step")

Vocab size:  71290  + UNK
Words per epoch:  17005207


In [6]:
print(x.eval())
print(y.eval())

[3134 3134 3134 3134 3134 3134 3134 3134   46   46   59   59   59   59   59
   59]
[  742   477   195     2  3134    46    59   128   742   477 10573   134
  3134    46    59   156]


In [7]:
# w_in : vocab_size x embed_dim
embed = tf.Variable(
    tf.random_uniform(
        [vocab_size, embed_dim],
        -0.5 / embed_dim,
        +0.5 / embed_dim
    ),
    name = "w_in"
)

# w_out : vocab_size x embed_dim
w = tf.Variable(tf.zeros([vocab_size, embed_dim]), name="w")
b = tf.Variable(tf.zeros([vocab_size]), name="b")

# Cast [batch_size] -> [batch_size x 1]
labels_matrix = tf.reshape(tf.cast(y, dtype=tf.int64), [batch_size, 1])

# Samples a set of classes using the provided (fixed) base distribution.
neg_sampled_ids, _, _ = (tf.nn.fixed_unigram_candidate_sampler(
    true_classes = labels_matrix,
    num_true = 1,
    num_sampled = num_neg_samples,
    unique = True,
    range_max = vocab_size,
    distortion = 0.75,
    unigrams = vocab_counts.tolist()))

# batch_size x embed_dim
x_emb = tf.nn.embedding_lookup(embed, x)

# batch_size x embed_dim
true_w = tf.nn.embedding_lookup(w, y)

# batch_size x 1
true_b = tf.nn.embedding_lookup(b, y)

# element-wise multiplication
# tf.reduced_sum([batch_size x embed_dim] * [batch_size x embed_dim]) + [batch_size x 1]
# tf.reduce_sum(x, 1) : row끼리 더함
true_y_ = tf.reduce_sum(tf.mul(x_emb, true_w), 1) + true_b


########################
# For negative samples
########################

# num_neg_samples x embed_dim
neg_w = tf.nn.embedding_lookup(w, neg_sampled_ids)

# num_neg_samples x 1
neg_b = tf.nn.embedding_lookup(b, neg_sampled_ids)
neg_b = tf.reshape(neg_b, [num_neg_samples])

# matrix multiplication
# [batch_size x embed_dim] x [embed_dim x num_neg_samples] + [num_neg_samples] <- 각 row 마다 neg_b 더함
# neg_y_ = tf.reduce_sum(tf.mul(x_emb, ))
neg_y_ = tf.matmul(x_emb, neg_w, transpose_b=True) + neg_b

# neg_y_ 계산이 이상해 보이지만 true_y_ 구할 때 처럼 element-wise multiplication해서 neg_b 더하는것

#####################################
# Negative Cross Entropy (NCE) Loss
#####################################

# Measures the probability error in discrete classification tasks in which each
# class is independent and not mutually exclusive. For instance, one could
# perform multilabel classification where a picture can contain both an elephant
# and a dog at the same time.
# >>> tmp = tf.maximum(y, 0) - y * y_ + tf.log(tf.ones_like(y) + tf.exp(-y))
# >>> [[ 0.69314718  0.7132616   0.69314718]]
# Range from 0 to 1
true_loss = tf.nn.sigmoid_cross_entropy_with_logits(true_y_, tf.ones_like(true_y_))
neg_loss = tf.nn.sigmoid_cross_entropy_with_logits(neg_y_, tf.zeros_like(neg_y_))

loss = (tf.reduce_sum(true_loss) + tf.reduce_sum(neg_loss)) / batch_size


#####################################
# Optimization
#####################################

global_step = tf.Variable(0, name="global_step")

words_to_train = float(words_per_epoch * epochs_to_train)

lr = learning_rate * tf.maximum(
    0.001,
    1.0 - tf.cast(total_words_processed, tf.float32) / words_to_train
)

optimizer = tf.train.GradientDescentOptimizer(lr)
train = optimizer.minimize(loss,
                           global_step = global_step,
                           gate_gradients=optimizer.GATE_NONE)

In [8]:
concurrent_steps = 12
statistics_interval = 5
checkpoint_interval = 600

init = tf.initialize_all_variables()
sess.run(init)
saver = tf.train.Saver()

with open(os.path.join("./", "vocab.txt"), "w") as f:
    for i in xrange(vocab_size):
        f.write(vocab_words[i] + " " + str(vocab_counts[i]) + "\n")

def train_thread_body():
    global sess, train, current_epoch
    initial_epoch = sess.run([current_epoch])
    
    while True:
        _, epoch = sess.run([train, current_epoch])
        if epoch != initial_epoch:
            break

###############
# Train
###############

for _ in xrange(epochs_to_train):
    initial_epoch, initial_words = sess.run([current_epoch, words])
    
    summary_op = tf.merge_all_summaries()
    summary_writer = tf.train.SummaryWriter("./",
                                            graph_def=sess.graph_def)
    
    workers = []
    for _ in xrange(concurrent_steps):
        t = threading.Thread(target=train_thread_body())
        t.start()
        workers.append(t)
        
    last_words, last_tme, last_summary_time = initial_words, time.time(), 0
    last_checkpoint_time = 0
    
    while True:
        # Print statistics every n seconds.
        time.sleep(statistics_interval)
        (epoch, step, loss, words, lr) = sess.run(
              [current_epoch, global_step, loss, words, lr])
        
        now = time.time()
        last_words, last_time, rate = words,now, (words - last_words) / (now - last_time)
        print("Epoch %4d Step %8d: lr = %5.3f loss = %6.2f words/sec = %8.0f\r" %
            (epoch, step, lr, loss, rate), end="")
        sys.stdout.flush()
        
        if now - last_checkpoint_time > checkpoint_interval:
            saver.save(sess, "./" + "model", global_step = step.astype(int))
            last_checkpoint_time = now
            
        if epoch != initial_epoch:
            break
            
    for t in workers:
        t.join()

# global_step : to distinguish step from other saved models
saver.save(sess, os.path.join("./", "model.ckpt"), global_step=model.global_step)

KeyboardInterrupt: 

In [ ]:
tmp = np.ones([3., 3.])
tmp[1][1] = 0
tmp[2][1] = 0
tmp[2][2] = 0

print(tmp)
print(tf.nn.l2_normalize(tmp.astype("float32"), 1).eval())

In [ ]:
########################
# Graph for evaluation
########################

analogy_a = tf.placeholder(dtype=tf.int32) # [N]
analogy_b = tf.placeholder(dtype=tf.int32) # [N]
analogy_c = tf.placeholder(dtype=tf.int32) # [N]

# [vocab_size x embed_dim]
normalized_embed = tf.nn.l2_normalize(embed, 1)

# [N x embed_dim]
a_embed = tf.gather(normalized_embed, analogy_a)
b_embed = tf.gather(normalized_embed, analogy_b)
c_embed = tf.gather(normalized_embed, analogy_c)

target = c_embed + (b_embed - a_embed)

# reference http://stats.stackexchange.com/questions/146221/is-cosine-similarity-identical-to-l2-normalized-euclidean-distance

# cosine(A,B) : angle between vector A and B (1: same, -1: totally different)
# Compute cosine distance between each pair of target and vocab.
# [N x embed_dim] * [embed_dim x vocab_size]  = [N x vocab_size]
dist = tf.matmul(target, normalized_embed, transpose_b = True)

_, pred_idx = tf.nn.top_k(dist, 4)


# Nodes for computing neighbors for a given word according to their cosine distance.
# [N]
nearby_word = tf.placeholder(dtype=tf.int32)

# [N x embed_dim]
nearby_embed = tf.gather(normalized_embed, nearby_word)

# [N x embed_dim] x [embed_dim x vocab_size] = [N x vocab_size]
nearby_dist = tf.matmul(nearby_emb, normalized_embed, transpose_b=True)

# Calculate top 1000 nearby index for each words
nearby_val, nearby_idx = tf.nn.top_k(nearby_dist,
                                     min(1000, vocab_size))

In [ ]:
tf.nn.top_k?

In [ ]:
embed = tf.Variable(
    tf.random_uniform(
        [5, 5],
        -0.5 / 5,
        +0.5 / 5
    ),
    name = "w_in"
)

tf.initialize_all_variables().run()

print(embed.eval())
print(tf.nn.embedding_lookup(embed, [2,1,4]).eval())
print(tf.gather(embed, [2,1,4]).eval())

# tf.nn.embedding_lookup(params, ids)

Looks up `ids` in a list of embedding tensors.

This function is used to perform parallel lookups on the list of
tensors in `params`.  It is a generalization of
[`tf.gather()`](../../api_docs/python/array_ops.md#gather), where `params` is
interpreted as a partition of a larger embedding tensor.

If `len(params) > 1`, each element `id` of `ids` is partitioned between
the elements of `params` by computing `p = id % len(params)`, and is
then used to look up the slice `params[p][id // len(params), ...]`.

The results of the lookup are then concatenated into a dense
tensor. The returned tensor has shape `shape(ids) + shape(params)[1:]`.

# tf.gather(params, indices)

Gather slices from `params` according to `indices`.

`indices` must be an integer tensor of any dimension (usually 0-D or 1-D).
Produces an output tensor with shape `indices.shape + params.shape[1:]` where:

    # Scalar indices
    output[:, ..., :] = params[indices, :, ... :]

    # Vector indices
    output[i, :, ..., :] = params[indices[i], :, ... :]

    # Higher rank indices
    output[i, ..., j, :, ... :] = params[indices[i, ..., j], :, ..., :]

If `indices` is a permutation and `len(indices) == params.shape[0]` then
this operation will permute `params` accordingly.


In [11]:
xx = embed.eval()
nembed = tf.nn.l2_normalize(embed, 1)

In [13]:
nearby_word = tf.placeholder(dtype=tf.int32)  # word id
nearby_emb = tf.reshape(tf.gather(nembed, nearby_word), [1, embed_dim])
nearby_dist = tf.matmul(nearby_emb, nembed, transpose_b=True)
nearby_val, nearby_idx = tf.nn.top_k(nearby_dist, min(50, vocab_size))

In [26]:
vocab_words[2]

'of'

In [15]:
vocab_words, vocab_counts, words_per_epoch

(array(['UNK', 'the', 'of', ..., 'jimsonweed', 'shinkansen', 'lophophore'], dtype=object),
 array([ 286363, 1061396,  593677, ...,       5,       5,       5], dtype=int32),
 17005207)

In [27]:
def nearby(idx, num=20):
    ids = np.array([x for x in idx])
    vals, idx = sess.run(
        [nearby_val, nearby_idx], {nearby_word: ids})
    for i, id in enumerate(idx):
        print("\n%s\n=====================================" % (vocab_words[id]))
        for (neighbor, distance) in zip(idx[i, :num], vals[i, :num]):
            print("%-20s %6.4f" % (vocab_words[neighbor], distance))

In [29]:
nearby([10], num=20)


['two' 'three' 'four' 'six' 'zero' 'one' 'five' 'eight' 'seven' 'june'
 'april' 'billion' 'january' 'february' 'march' 'july' 'december' 'mm'
 'august' 'nine' 'september' 'kg' 'km' 'november' 'bwv' 'october' 'ft' 'mi'
 'million' 'lb' 'retrieved' 'cm' 'inches' 'miles' 'sq' 'summer' 'unpaved'
 'pp' 'imports' 'nautical' 'paved' 'accessed' 'feet' 'mhz' 'nm' 'cubic'
 'airports' 'finalist' 'square' 'quarter']
two                  1.0000
three                0.9548
four                 0.9408
six                  0.9262
zero                 0.9261
one                  0.9229
five                 0.9229
eight                0.9113
seven                0.9078
june                 0.8708
april                0.8667
billion              0.8658
january              0.8655
february             0.8644
march                0.8630
july                 0.8630
december             0.8617
mm                   0.8568
august               0.8566
nine                 0.8565
